In [26]:
# Testing variations of OLD MODEL to check how training and epochs affect the changes in
# weights assuming we start from the same random initialization

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, Flatten, Activation, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

from tensorflow.keras.datasets import mnist
import keras

OLD_MODEL_EPOCHS = 60
OLD_MODEL_DIR = "redundancy_check"
OLD_MODEL_VERSION = "2"

def old_model_name(suffix=None, teacher=False, student=False):
    typ = "base"
    if teacher:
        typ = "teacher"
    if student:
        typ = "student"
    if suffix is not None:
        return "{}/v{}_{}_old_{}_{}".format(OLD_MODEL_DIR, OLD_MODEL_VERSION, typ, OLD_MODEL_EPOCHS, suffix)
    else:
        return "{}/v{}_{}_old_{}".format(OLD_MODEL_DIR, OLD_MODEL_VERSION, typ, OLD_MODEL_EPOCHS)

#https://www.kaggle.com/vincentman0403/mnist-classification-by-cnn-0-996-top-8#3.-Set-up-CNN-model
def make_model_old():
    model = Sequential()
    model.add(Conv2D(filters=36,
                     kernel_size=(3, 3),
                     padding='same',
                     input_shape=(28, 28, 1),
                     activation='relu'))
    model.add(Conv2D(filters=36,
                     kernel_size=(3, 3),
                     padding='same',
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(filters=36,
                     kernel_size=(3, 3),
                     padding='same',
                     activation='relu'))
    model.add(Conv2D(filters=36,
                     kernel_size=(3, 3),
                     padding='same',
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    
    # Adding another group v1
    model.add(Conv2D(filters=36,
                     kernel_size=(3, 3),
                     padding='same',
                     activation='relu'))
    model.add(Conv2D(filters=36,
                     kernel_size=(3, 3),
                     padding='same',
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    
#     # Adding another group v3
#     model.add(Conv2D(filters=36,
#                      kernel_size=(3, 3),
#                      padding='same',
#                      activation='relu'))
#     model.add(Conv2D(filters=36,
#                      kernel_size=(3, 3),
#                      padding='same',
#                      activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))

    
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(units=512))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(10, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

def train_model_old(model, x_train, y_train, name):
    learning_rate_reduction = ReduceLROnPlateau(monitor='accuracy', patience=3, verbose=1, factor=0.5, min_lr=0.00001)
    # Use the other models thing instead v2
    annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** (x+OLD_MODEL_EPOCHS))
    filepath = name + "-{epoch:02d}"
    checkpoint = ModelCheckpoint(filepath, verbose=1, save_best_only=False, mode='auto', save_freq='epoch')
    
    batch_size = 32
    epochs = OLD_MODEL_EPOCHS # Was 30, 50
    train_history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, steps_per_epoch=x_train.shape[0] // batch_size, callbacks=[annealer, checkpoint])

In [27]:
# Generate the train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)
 
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))
 
# Generate some noisy data
noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)
 
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

In [28]:
t = make_model_old()
t.save(old_model_name())
s = tf.keras.models.load_model(old_model_name())

INFO:tensorflow:Assets written to: redundancy_check/v2_base_old_60/assets


In [29]:
train_model_old(t, x_train, y_train, old_model_name(teacher=True))
t.save(old_model_name(teacher=True))

Epoch 1/60
1875/1875 [==============================] - ETA: 0s - loss: 0.5205 - accuracy: 0.8432
Epoch 00001: saving model to redundancy_check/v2_teacher_old_60-01
INFO:tensorflow:Assets written to: redundancy_check/v2_teacher_old_60-01/assets
1875/1875 [==============================] - 92s 49ms/step - loss: 0.5205 - accuracy: 0.8432
Epoch 2/60
1875/1875 [==============================] - ETA: 0s - loss: 0.1347 - accuracy: 0.9606
Epoch 00002: saving model to redundancy_check/v2_teacher_old_60-02
INFO:tensorflow:Assets written to: redundancy_check/v2_teacher_old_60-02/assets
1875/1875 [==============================] - 101s 54ms/step - loss: 0.1347 - accuracy: 0.9606
Epoch 3/60
1875/1875 [==============================] - ETA: 0s - loss: 0.0937 - accuracy: 0.9719
Epoch 00003: saving model to redundancy_check/v2_teacher_old_60-03
INFO:tensorflow:Assets written to: redundancy_check/v2_teacher_old_60-03/assets
1875/1875 [==============================] - 104s 55ms/step - loss: 0.0937 - a

In [30]:
y = t.predict(x_train)
y = np.argmax(y, axis=1)
import keras

t_y_test = keras.utils.to_categorical(y)
train_model_old(s, x_train, t_y_test, old_model_name(student=True))
s.save(old_model_name(student=True))

Epoch 1/60
1875/1875 [==============================] - ETA: 0s - loss: 0.5175 - accuracy: 0.8425
Epoch 00001: saving model to redundancy_check/v2_student_old_60-01
INFO:tensorflow:Assets written to: redundancy_check/v2_student_old_60-01/assets
1875/1875 [==============================] - 116s 62ms/step - loss: 0.5175 - accuracy: 0.8425
Epoch 2/60
1875/1875 [==============================] - ETA: 0s - loss: 0.1316 - accuracy: 0.9611
Epoch 00002: saving model to redundancy_check/v2_student_old_60-02
INFO:tensorflow:Assets written to: redundancy_check/v2_student_old_60-02/assets
1875/1875 [==============================] - 117s 63ms/step - loss: 0.1316 - accuracy: 0.9611
Epoch 3/60
1875/1875 [==============================] - ETA: 0s - loss: 0.0903 - accuracy: 0.9727
Epoch 00003: saving model to redundancy_check/v2_student_old_60-03
INFO:tensorflow:Assets written to: redundancy_check/v2_student_old_60-03/assets
1875/1875 [==============================] - 110s 59ms/step - loss: 0.0903 - 

In [31]:
t.evaluate(x_test, y_test)
s.evaluate(x_test, y_test)

313/313 [==============================] - 5s 15ms/step - loss: 0.0187 - accuracy: 0.9941


[0.018657786771655083, 0.9940999746322632]